In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:


import pandas as pd
import numpy as np
from privacypacking.utils.utils import LOGS_PATH
from experiments.ray.analysis import load_ray_experiment,load_latest_scheduling_results
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import copy

from collections import defaultdict

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-16_14-34-27")
df = load_ray_experiment(logs)
# df["alpha"] = df.config.apply(lambda x: x["omegaconf"]["alphas"][0])

In [ ]:
px.bar(
    df,
    x="scheduler_metric",
    y="realized_profit",
    title="Profit<br> Offline multiblock, PrivateKube",
    width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-16_14-39-10")
df = load_ray_experiment(logs)

In [ ]:
df.temperature

In [ ]:
px.line(
    df.sort_values("temperature"),
    x="temperature",
    y="realized_profit",
    title="Profit<br> Offline multiblock, PrivateKube",
    log_x=True,
    width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-16_14-47-53")
df = load_ray_experiment(logs)
px.line(
    df.sort_values("temperature"),
    x="temperature",
    y="realized_profit",
    title="Profit<br> Offline multiblock, PrivateKube",
    log_x=True,
    width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-16_19-08-49")
df = load_ray_experiment(logs)
px.line(
    df.sort_values("temperature"),
    x="temperature",
    y="realized_profit",
    title="Profit<br> Offline multiblock, PrivateKube",
    log_x=True,
    width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-18_18-47-28")
df = load_ray_experiment(logs)
px.line(
    df.sort_values("temperature"),
    x="temperature",
    y="realized_profit",
    title="Profit<br> Online multiblock, PrivateKube. Softmax with numerical error.",
    log_x=True,
    # width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-18_17-13-10")
df = load_ray_experiment(logs)
px.line(
    df.sort_values("temperature"),
    x="temperature",
    y="realized_profit",
    title="Profit<br> Online multiblock, PrivateKube. Correct softmax.",
    log_x=True,
    range_y=[0.95e6,1e6]
    # width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-18_19-20-53")
df = load_ray_experiment(logs)
px.bar(
    df.sort_values("realized_profit"),
    x="scheduler_metric",
    y="realized_profit",
    title="Profit<br> Online multiblock, PrivateKube. Correct softmax.",
    # log_x=True,
    width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-16_16-20-47")
df = load_ray_experiment(logs)
px.bar(
    df,
    x="scheduler_metric",
    y="realized_profit",
    title="Profit<br> Offline multiblock, PrivateKube. Broken softmax.",
    # log_x=True,
    width=800,
)

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-16_16-20-47")
df = load_ray_experiment(logs)
df
# px.bar(
#     df,
#     x="scheduler_metric",
#     y="realized_profit",
#     title="Profit<br> Offline multiblock, PrivateKube",
#     log_x=True,
#     width=800,
# )

In [ ]:
px.bar(
    df,
    x="scheduler_metric",
    y="realized_profit",
    title="Profit<br> Offline multiblock, PrivateKube",
    # log_x=True,
    width=800,
)

In [ ]:
# TODO: get the results on multiple alphas
# logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-09_16-01-07")
df = load_ray_experiment(logs)
df["alpha"] = df.config.apply(lambda x: x["omegaconf"]["alphas"][0])

In [ ]:
df

In [ ]:
sdf = load_latest_scheduling_results(expname="ray/run_and_report_2022-02-10_09-38-20", alphas="alphas", verbose=True)
# run_and_report_2022-02-03_10-01-17

In [ ]:
sdf.head()

In [ ]:
dominant_shares = sdf.query("metric == 'Fcfs'").groupby(["id"])["normalized_epsilon"].agg(max)
dominant_shares = pd.DataFrame(dominant_shares).reset_index()
dominant_shares["dominant_shares"] = dominant_shares["normalized_epsilon"]
dominant_shares= dominant_shares.drop("normalized_epsilon", axis=1)


In [ ]:
dominant_shares.describe()

In [ ]:
merged = pd.merge(sdf, dominant_shares, on=["id"])

In [ ]:
merged.columns

# Fraction of fair pipelines

In [ ]:
merged.head()

In [ ]:
merged.describe()

In [ ]:
# tasks = merged.groupby(["id","alpha", "metric"])["id", "alpha", "metric", "dominant_shares","allocated"]
tasks = merged.groupby(["id","metric"]).first()
tasks = tasks.reset_index()
tasks = tasks[["id", "metric", "dominant_shares","allocated"]]

In [ ]:
tasks.head(20)

In [ ]:
d = defaultdict(list)
for N in range(1,200,1):
    # for alpha in [4,16,64]:
    total_fair_tasks = tasks.query(f"dominant_shares <= {1/N} and metric == 'Fcfs'")["id"].count()

    d["N"].append(N)
    # d["alpha"].append(alpha)
    d["metric"].append("total")
    d["n_fair_tasks"].append(total_fair_tasks)


    d["N"].append(N)
    # d["alpha"].append(alpha)
    d["metric"].append("N")
    d["n_fair_tasks"].append(N)

    for metric in ["DynamicFlatRelevance", "Fcfs", "Fcfs", "DominantShares", "SoftKnapsack"]:
        allocated_fair_tasks = tasks.query(f"dominant_shares <= {1/N} and metric == '{metric}' and allocated")["id"].count()

        d["N"].append(N)
        # d["alpha"].append(alpha)
        d["metric"].append(metric)
        d["n_fair_tasks"].append(allocated_fair_tasks)
            
fair_tasks = pd.DataFrame(d)

In [ ]:
px.line(
    fair_tasks,
    x="N",
    y="n_fair_tasks",
    # range_y=[0,20],
#     color="log_id",
    color="metric",
    # barmode="group",
    # pattern_shape="allocated",
#     log_x=True,
    # range_x=[0,20],
    # range_y=[1e-3, 1e5],
    # facet_col="alpha",
    # facet_col_wrap=1,
    # height=1200,
    width=800,
    title="Number of fair tasks allocated"
#     animation_frame="id"
)